In [21]:
#from Covid_19.web_page import plotty

import cv2



In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

In [22]:
#from Covid_19.web_page import uploaded_file



In [23]:
from joblib import load

In [29]:
#loaded_model = load('../notebooks/model2.joblib')

In [3]:
!pwd




/home/leesalem/code/NinLev/Covid_19/notebooks


In [34]:
def plot_loss_accuracy(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()

In [41]:
counter = random.randrange(10000)
list_of_names = []
for i in range(5):
    counter +=1
    image_name = f"image_numer({counter})"
    list_of_names.append(image_name)
    

In [42]:
list_of_names


['image_numer(1599)',
 'image_numer(1600)',
 'image_numer(1601)',
 'image_numer(1602)',
 'image_numer(1603)']

In [14]:
import random

In [16]:
random.randrange(999999) 

974187

## MODELING 


In [1]:

import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow.keras import callbacks


In [2]:
def read_ct_img_bydir(target_dir):
    img=cv2.imdecode(np.fromfile(target_dir,dtype=np.uint8),cv2.IMREAD_GRAYSCALE)
    #img = corp_margin(img)
    img=cv2.resize(img,(200,200))
    
    return img


In [9]:
!pwd


/home/leesalem/code/NinLev/Covid_19/notebooks


In [26]:
def get_data():
    '''Loads raw data from image names of each class folder and assigns a label to it.'''
    
    target_dir1='../../../raw_data/labeled_CT_data/NiCT/'
    target_dir2='../../../raw_data/labeled_CT_data/pCT/'
    target_dir3='../../../raw_data/labeled_CT_data/nCT/'

    target_list1=[target_dir1+file for file in os.listdir(target_dir1)]
    target_list2=[target_dir2+file for file in os.listdir(target_dir2)]
    target_list3=[target_dir3+file for file in os.listdir(target_dir3)]

    target_list=target_list1+target_list2+target_list3

    # Assign labels: 0: 'non informative CT'
    #                1: 'positive Covid-19 CT'
    #                2: 'negative Covid-19 CT'
    y_list=to_categorical(np.concatenate(np.array([[0]*len(target_list1),
                                               [1]*len(target_list2),
                                               [2]*len(target_list3)])),3)

    X=np.array([read_ct_img_bydir(file) for file in target_list])[:,:,:,np.newaxis]

    return X, y_list

In [27]:
X,y = get_data()


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [29]:
def model_init():
    from tensorflow.keras.models import Sequential
    from tensorflow.keras import layers

    model = Sequential()
    model.add(layers.Conv2D(64, (3,3), input_shape=(200, 200, 1), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 

    model.add(layers.Conv2D(128, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Conv2D(64, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 

    model.add(layers.Conv2D(32, (3,3), activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2,2))) 

    model.add(layers.Dense(32, activation="relu"))

    model.add(layers.Flatten())
    model.add(layers.Dense(3, activation="softmax"))

    # The compilation
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', metrics = 'accuracy')  # Recommanded optimizer for RNN
    return model


In [33]:
salem = model_init()

es = callbacks.EarlyStopping(patience=20, restore_best_weights=True)

history = salem.fit(X_train, y_train,
                    batch_size=32,
                    epochs=1,
                    validation_split=0.3,
                    callbacks=[es],
                    verbose=2)

216/216 - 999s - loss: 0.9896 - accuracy: 0.8045 - val_loss: 0.2274 - val_accuracy: 0.9238


In [ ]:
## The prediction
y_pred = model.predict(X_test)

In [ ]:
y_pred_class = np.rint(y_pred)

In [ ]:
y_pred_class

In [ ]:
pred = model.evaluate(X_test, y_test, verbose=1)

In [ ]:
# save model
model.save('salem')  # creates a HDF5 file 'my_model.h5'


In [ ]:
plot_loss_accuracy(history)

In [ ]:
g=plt.plot(y_test - y_pred,marker='o',linestyle='')
